In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [2]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
from openpyxl_image_loader import SheetImageLoader

In [9]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [8]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def get_sign(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [10]:
!cat .env

STAGE=PRODUCTION


In [11]:
filename='Terraplenos_estruturas_contencao'

In [12]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
# sheetname
ws = wb[sheetname]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [13]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
#     if obj['Rodovia'] is not None:           
#         obj['img'] = [a for a in ws._images if a.anchor._from.row == index]
#         for i, img in enumerate(obj['img']):
#             if i == 0:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 1_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 1_1']
#             elif i == 1:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 1_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 1_2']
#             elif i == 2:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 2_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 2_1']
#             elif i == 3:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 2_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 2_2']
#             elif i == 4:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 3_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 3_1']
#             elif i == 5:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 3_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 3_2']
#             elif i == 6:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 4_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 4_1']
#             elif i == 7:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 4_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 4_2']

        
    values.append(obj)

In [14]:
values[0]

{'km': 0,
 'km final': 10,
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'Civil: SGTEC',
 'Encontrado em': datetime.datetime(2023, 1, 1, 0, 0),
 'Sentido': 'Sul',
 'Classe': 'Terraplenos e Estrutura de Contenção',
 'Faixa': 'Faixa 2',
 'Rodovia': 'BR-116 SP',
 'Latitude (Grau decimal - 00,00000)': -10,
 'Longitude  (Grau decimal - 00,00000)': -20,
 'X UTM': 30,
 'Y UTM': 40,
 'Zona UTM': 'zona tal',
 'Identificação': 'identificado',
 'UF': 'rs',
 'Extensão Terrapleno (m)': 10000,
 'Altura Terrapleno (m)': 20000,
 'Inclinação Terrapleno (°)': 30000,
 'Distância Acostamento Terrapleno (m)': 40000,
 'Tipo Terrapleno': 'Corte',
 'Tipo de Relevo': 'Suave',
 'Vegetação': 'Arbustiva',
 'Densidade da Vegetação': 'Esparça',
 'Drenagem Superficial': 'Construída',
 'Tipo de Drenagem Superficial': 'Sarjetas',
 'Drenagem Subterrânea': 'Natural',
 'Tipo de Drenagem Subterrânea': 'DHP (Dreno Horizontal Profundo)',
 'Passivo Ambiental': 'SIM',
 'Descrição do Passivo Ambiental': 'desc passivo tal',

In [17]:
company = Company.objects.get(name='CCR - Rio Sp')

occurrence_type=OccurrenceType.objects.get(uuid='62b15386-f114-44ff-8879-a561fbe1c460')
user=User.objects.get(username='rlcs')
utc = pytz.timezone('America/Sao_Paulo')
company,occurrence_type

(<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>,
 <OccurrenceType: Terraplenos e Estrutura de Contenção - ['CCR - Rio Sp']>)

In [18]:
for a in tqdm(values):
    a['contencao'] = list(filter(None, [{'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_1']) if a['Tipo de Estrutura_1'] is not None else None, 'other': a['Outro tipo de estrutura_1'], 'extenstion': a['Extensão Contenção (m)_1'], 'inner_height': a['Altura Contenção (m)_1'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_1']) if a['Ancoragens da Contenção_1'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_1']} if a['Tipo de Estrutura_1'] is not None else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_2']) if a['Tipo de Estrutura_2'] is not None else None, 'other': a['Outro tipo de estrutura_2'], 'extenstion': a['Extensão Contenção (m)_2'], 'inner_height': a['Altura Contenção (m)_2'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_2']) if a['Ancoragens da Contenção_2'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_2']} if a['Tipo de Estrutura_2'] is not None else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_3']) if a['Tipo de Estrutura_3'] is not None else None, 'other': a['Outro tipo de estrutura_3'], 'extenstion': a['Extensão Contenção (m)_3'], 'inner_height': a['Altura Contenção (m)_3'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_3']) if a['Ancoragens da Contenção_3'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_3']} if a['Tipo de Estrutura_3'] is not None else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_4']) if a['Tipo de Estrutura_4'] is not None else None, 'other': a['Outro tipo de estrutura_4'], 'extenstion': a['Extensão Contenção (m)_4'], 'inner_height': a['Altura Contenção (m)_4'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_4']) if a['Ancoragens da Contenção_4'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_4']} if a['Tipo de Estrutura_4'] is not None else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_5']) if a['Tipo de Estrutura_5'] is not None else None, 'other': a['Outro tipo de estrutura_5'], 'extenstion': a['Extensão Contenção (m)_5'], 'inner_height': a['Altura Contenção (m)_5'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_5']) if a['Ancoragens da Contenção_5'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_5']} if a['Tipo de Estrutura_5'] is not None else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_6']) if a['Tipo de Estrutura_6'] is not None else None, 'other': a['Outro tipo de estrutura_6'], 'extenstion': a['Extensão Contenção (m)_6'], 'inner_height': a['Altura Contenção (m)_6'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_6']) if a['Ancoragens da Contenção_6'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_6']} if a['Tipo de Estrutura_6'] is not None else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_7']) if a['Tipo de Estrutura_7'] is not None else None, 'other': a['Outro tipo de estrutura_7'], 'extenstion': a['Extensão Contenção (m)_7'], 'inner_height': a['Altura Contenção (m)_7'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_7']) if a['Ancoragens da Contenção_7'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_7']} if a['Tipo de Estrutura_7'] is not None else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_8']) if a['Tipo de Estrutura_8'] is not None else None, 'other': a['Outro tipo de estrutura_8'], 'extenstion': a['Extensão Contenção (m)_8'], 'inner_height': a['Altura Contenção (m)_8'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_8']) if a['Ancoragens da Contenção_8'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_8']} if a['Tipo de Estrutura_8'] is not None else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_9']) if a['Tipo de Estrutura_9'] is not None else None, 'other': a['Outro tipo de estrutura_9'], 'extenstion': a['Extensão Contenção (m)_9'], 'inner_height': a['Altura Contenção (m)_9'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_9']) if a['Ancoragens da Contenção_9'] is not None else None, 'inner_quantity': a['Quantidade (unidade)_9']} if a['Tipo de Estrutura_9'] is not None else None,]))

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
values[0]

{'km': 0,
 'km final': 10,
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'Civil: SGTEC',
 'Encontrado em': datetime.datetime(2023, 1, 1, 0, 0),
 'Sentido': 'Sul',
 'Classe': 'Terraplenos e Estrutura de Contenção',
 'Faixa': 'Faixa 2',
 'Rodovia': 'BR-116 SP',
 'Latitude (Grau decimal - 00,00000)': -10,
 'Longitude  (Grau decimal - 00,00000)': -20,
 'X UTM': 30,
 'Y UTM': 40,
 'Zona UTM': 'zona tal',
 'Identificação': 'identificado',
 'UF': 'rs',
 'Extensão Terrapleno (m)': 10000,
 'Altura Terrapleno (m)': 20000,
 'Inclinação Terrapleno (°)': 30000,
 'Distância Acostamento Terrapleno (m)': 40000,
 'Tipo Terrapleno': 'Corte',
 'Tipo de Relevo': 'Suave',
 'Vegetação': 'Arbustiva',
 'Densidade da Vegetação': 'Esparça',
 'Drenagem Superficial': 'Construída',
 'Tipo de Drenagem Superficial': 'Sarjetas',
 'Drenagem Subterrânea': 'Natural',
 'Tipo de Drenagem Subterrânea': 'DHP (Dreno Horizontal Profundo)',
 'Passivo Ambiental': 'SIM',
 'Descrição do Passivo Ambiental': 'desc passivo tal',

In [44]:
objects=[]

for index, a in enumerate(tqdm(values)):

    point, road = get_road_coordinates(a['Rodovia'], a['km'], str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
    if road.name != a['Rodovia']:
        print(road.name,a['Rodovia'])
        
    try:
        objects.append(Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=get_connected_reference(company, 'reporting', 'lane', a['Faixa']),
                    road=road,
                    road_name=road.name,
                    direction=get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize()),
                    created_by=user,
#                     found_at=datetime.datetime(2023, 5, 16, 0, 0).replace(tzinfo=utc),
                    km=a['km'],
                    end_km=a['km final'],
                    point=point,
                    form_data={
                        'latitude': a['Latitude (Grau decimal - 00,00000)'],
                        'longitude': a['Longitude  (Grau decimal - 00,00000)'],
                        'x_utm': a['X UTM'],
                        'y_utm': a['Y UTM'],
                        'zona_utm': a['Zona UTM'],
                        'id_ccr_antt': a['Identificação ANTT'],
                        'uf': a['UF'],
                        'length': a['Extensão Terrapleno (m)'],
                        'height_terrapleno': a['Altura Terrapleno (m)'],
                        'inclination': a['Inclinação Terrapleno (°)'],
                        'distance': a['Distância Acostamento Terrapleno (m)'],
                        'tipo_terrapleno': get_value('Tipo Terrapleno',occurrence_type,a['Tipo Terrapleno']) if a['Tipo Terrapleno'] is not None else None,
                        'terrain_embossing': get_value('Tipo de Relevo',occurrence_type,a['Tipo de Relevo']) if a['Tipo de Relevo'] is not None else None,
                        'vegetation': get_value('Vegetação',occurrence_type,a['Vegetação']) if a['Vegetação'] is not None else None,
                        'vegetation_density': get_value('Densidade da Vegetação',occurrence_type,a['Densidade da Vegetação']) if a['Densidade da Vegetação'] is not None else None,
                        'is_contention': True if a['Existe contenção?'].lower() == 'sim' else False,
                        'contention': a['contencao'],
                        'drenagem_superficial': get_value('Drenagem Superficial',occurrence_type,a['Drenagem Superficial']) if a['Drenagem Superficial'] is not None else None,
                        'dre_kind_sup': get_value('Tipo de Drenagem Superficial',occurrence_type,a['Tipo de Drenagem Superficial']) if a['Tipo de Drenagem Superficial'] is not None else None,
                        'drenagem_subterranea': get_value('Drenagem Subterrânea',occurrence_type,a['Drenagem Subterrânea']) if a['Drenagem Subterrânea'] is not None else None,
                        'kind_drenagem_subterranea': get_value('Tipo de Drenagem Subterrânea',occurrence_type,a['Tipo de Drenagem Subterrânea']) if a['Tipo de Drenagem Subterrânea'] is not None else None,
                        'passivo_ambiental': True if a['Passivo Ambiental'].lower() == 'sim' else False,
                        'environment_description': a['Descrição do Passivo Ambiental'],
                        'notes': a['Observações'],
                        'index': a['Índice (Serial ANTT)']
                        },
                    ))
    except Exception as e:
        print(f'AQUI: km: {a["km"]} | Rodovia: {a["Rodovia"]} | Linha da Planilha: {index+2} | {e}')
        pass

  0%|          | 0/106 [00:00<?, ?it/s]

In [45]:
objects[105].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f9933960d90>,
 'uuid': UUID('baf33d1d-4c6c-4a91-8a61-0c3d6c8cb1f3'),
 'number': '',
 'company_id': UUID('ec364a8e-b335-4729-8ac8-50ccabd73459'),
 'road_name': 'SP-255',
 'road_id': 1919,
 'km': 76.1,
 'end_km': None,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f99339507f0>,
 'direction': '1',
 'lane': '1',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('7c44852b-84d8-453f-bfc9-d3e2e92602b2'),
 'form_data': {'cod_acesso': 'SP255 - 44 - S',
  'cod_novo': None,
  'lote': 29,
  'tipo_da_obra': 'Duplicação Km 48+100 ao Km 77+100',
  'localization': 76.1,
  'latitude': -21.78692,
  'longitude': -48.121463,
  'city': '3',
  'interessado': 'Grupo Molina',
  'matricula': '118224',
  'cri': '1º de Araraquara',
  'tipo_acesso': '3',
  'classificacao': '2',
  'atividade_do_acesso': '1',

In [47]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/106 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00118
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00119
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00120
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00121
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00122
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00123
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2

Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00179
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00180
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00181
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00182
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00183
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00184
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 1919: SP-255
VPT-IT-2023.00185
'NoneType' object has no attribute 'order'
Searching KM on directio

In [43]:
# for reporting, images in tqdm(objects):
#     for image in reversed(images):
#         reporting_file = ReportingFile(
#             created_by=user,
#             reporting=reporting,
#             description=image.name,
#             km=reporting.km,
#             point=reporting.point
#         )
#         reporting_file.save()
#         image_io = BytesIO(image._data())
#         im = Image.open(image_io)
#         if im.mode in ("RGBA", "P"):
#             im = im.convert("RGB")
#         thumb_io = BytesIO()
#         im.save(thumb_io, format='jpeg', quality=90)
#         reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/585 [00:00<?, ?it/s]